# 本セクションの目次

1. コンポーネントの起動
2. データの送信と確認
3. データの送信と確認(コンソール)
4. データの送信と確認(ファイル)
4. データの送信と確認(メッセージ)
4. データの送信と確認(メモリ)

# コンポーネントの起動
既に起動はしているのですが、一度関係しているコンポーネントを覗いてみましょう。

## kafkaのトピックを作成します
今回は3つのトピックを作成します

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic1 \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka:9092 
```

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic2 \
    --replication-factor 1 \
    --partitions 1 \
    --bootstrap-server kafka:9092 
```

```
/home/pyspark/kafka/bin/kafka-topics.sh \
    --create --topic pyspark-topic5 \
    --replication-factor 1 \
    --partitions 3 \
    --bootstrap-server kafka:9092 
```

## replication-factor/partitions　
いわゆる冗長化の設定。
レプリケーションは何台に
パーティションは何個？
という意味の冗長化。

つまりreplication-factor 2でpartitons3であれば2台のノードにまたがってパーティションを3つ（1台に２つ、もう一台に1つ）  
という設定になる。

## bootstrap-server 
データの受付口（ブローカーのIPとポートを指定する）


# データの送信と確認

データの送信方法は何種類か存在します。

どのような場面でも対応できるように本コースでは以下の3つを対象とします。  
プログラムから送信するパターンとWeb画面から送信するパターンはのちのセクションにて別途抜き出して解説します。

- コマンドライン
- プログラムからデータを送信する
- Web画面から送信する

またデータの確認方法としてシンクを紹介します

- コンソールシンク
- メモリシンク
- メッセージシンク
- ファイルシンク

を紹介します。

In [1]:
# コンソールで設定したSparkとNoteBookを接続します(動かす前に毎度実行する必要があります)
import findspark
findspark.init("/home/pyspark/spark")

In [2]:
#pysparkに必要なライブラリを読み込む
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SparkSession

#spark sessionの作成
# spark.ui.enabled trueとするとSparkのGUI画面を確認することができます
# spark.eventLog.enabled true　とすると　GUIで実行ログを確認することができます
# GUIなどの確認は最後のセクションで説明を行います。
spark = SparkSession.builder \
    .appName("chapter1") \
    .config("hive.exec.dynamic.partition", "true") \
    .config("hive.exec.dynamic.partition.mode", "nonstrict") \
    .config("spark.sql.session.timeZone", "JST") \
    .config("spark.ui.enabled","true") \
    .config("spark.eventLog.enabled","true") \
    .config("spark.jars.packages", "org.apache.spark:spark-streaming_2.13:3.2.2,org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.2,org.apache.spark:spark-avro_2.12:3.2.2") \
    .enableHiveSupport() \
    .getOrCreate()

# パッケージを複数渡したい時は「,」で繋いで渡します。
# Sparkのバージョンにしっかりと合わせます(今回はSparkのバージョンが3.2.2を使っています。)。

:: loading settings :: url = jar:file:/home/pyspark/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/pyspark/.ivy2/cache
The jars for the packages stored in: /home/pyspark/.ivy2/jars
org.apache.spark#spark-streaming_2.13 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f35659d7-ccbc-409d-b0bb-cddb6f0ec4a0;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in ce

# kafkaにデータを送信してそのデータをコンソールに出してみよう

まずは最も単純にkafkaにデータをコマンドラインで送信しそのデータを読み取ってconsole出力してみましょう

cmd -> kafka <- spark streaming(df) -> console

In [3]:
# PySparkをKafkaと接続します(kafka <- spark streaming(df))
# ストリームの経路を作成行います。

# kafkaからデータを読み取る設定を行います。

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic1") \
  .load()

In [4]:
# ストリームから読み取って出力する設定(spark streaming(df) -> console)

console_stream = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .trigger(processingTime="5 seconds") \
  .format("console") \
  .option("checkpointLocation", "/tmp/kafka/console/") \
  .start()

# ストリームから読み取ったデータのうちkeyとvalueを抜き出してStringにキャストしたのちconsoleに出力してね
# triggerは何秒単位で処理を行うか（処理というのは↑のconsoleに出力する処理のこと）今回は動画の都合で短め(おおよそ1~10分が多い)
# checkpointlocationは、処理をすると「ここまで処理をしたよ！」という情報が出力される場所のこと
# 仮にkafkaが落ちてもチェックポイントのデータが生きていれば復旧した後に途中からやり直しをすることが可能になる。

#root
# |-- key: binary (nullable = true)   
# |-- value: binary (nullable = true)
# |-- topic: string (nullable = true)
# |-- partition: integer (nullable = true)
# |-- offset: long (nullable = true)
# |-- timestamp: timestamp (nullable = true)
# |-- timestampType: integer (nullable = true)

# binary型なのは様々な型のデータが入ってくることが想定されるから

21/12/12 01:58:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
|  1|    2|
|  2|    1|
+---+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
|  3|    1|
+---+-----+



# 送信コマンド(cmd -> kafka)

```

/home/pyspark/kafka/bin/kafka-console-producer.sh \
   --topic pyspark-topic1  \
   --bootstrap-server kafka:9092 \
   --property parse.key=true --property key.separator=,

```

key.separatorはキーバリューを分割するためのオプションです、今回はカンマで区切るとキーとバリューを送ることが可能です。


In [5]:
# ストリームの停止
console_stream.stop()

# ファイルシンクしてみよう
次は、コンソールではなくファイル内にシンクをしてみましょう

cmd -> kafka <- spark streaming(df) -> parquet(ファイル)

In [6]:
# ストリームから読み取って出力する設定(spark streaming(df) -> file)

file_stream = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("parquet") \
  .option("path", "/tmp/data/") \
  .outputMode("append") \
  .partitionBy("key") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/parquet/") \
  .start()

# ほぼフルフルでのオプションです(必要に応じてオプションを付け足したりします)
# formatはparqeut
# pathはどこに出力するか
# append(追記か？)overwrite(上書きか)
# partitionbyパーティションを付ける場合
# 読み込みの状況を途中保存するためのチェックポイント

# 単純なテキストがparquetに変換されて出力されます。

21/12/12 02:02:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
file_stream.stop()

In [7]:
read_df=spark.read.parquet("/tmp/data/*/*")
read_df.show()

+-----+
|value|
+-----+
|    1|
|    1|
|    6|
+-----+



# メッセージシンクをしてみよう
メッセージシンクとはメッセージキューから読み取ったデータを別のメッセージキューに送信することです  
cmd -> kafka(pyspark-topic1) <- spark streaming(df) -> kafka(pyspark-topic2)  <- spark streaming(df2)(データの確認のため) -> console

In [9]:
# データのチェックを行うためのストリームを生成します

# PySparkをKafkaと接続します(kafka(pyspark-topic2)  <- spark streaming(df2)(データの確認のため))
# ストリームの経路を作成行います。

# kafkaからデータを読み取る設定を行います。

df2 = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("subscribe", "pyspark-topic2") \
  .load()

In [10]:
# PySparkをKafkaと接続します(<- spark streaming(df2)(データの確認のため) -> console)
# ストリームの経路を作成行います。

# kafkaからデータを読み取る設定を行います。

console_stream_check = df2 \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .trigger(processingTime="5 seconds") \
  .format("console") \
  .option("checkpointLocation", "/tmp/kafka/console_check/") \
  .start()

21/12/12 02:04:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+



In [11]:
# ストリームから読み取ってメッセージシンクする設定(spark streaming(df) -> kafka(pyspark-tpoic2))

kafka_stream = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka:9092") \
  .option("topic", "pyspark-topic2") \
  .trigger(processingTime="5 seconds") \
  .option("checkpointLocation", "/tmp/kafka/pyspark-topic2/") \
  .start()

# ほぼフルフルでのオプションです(必要に応じてオプションを付け足したりします)
# formatはkafka
# ブローカーのサーバのIPを指定
# どのトピックに対して書き込みを行うか記載
# 読み込みの状況を途中保存するためのチェックポイント

21/12/12 02:06:41 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 1
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
| 34|   11|
+---+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
| 22|   11|
| 11|   23|
+---+-----+



In [12]:
console_stream_check.stop()
kafka_stream.stop()

# メモリシンクをしてみよう
メモリシンクとはその名の通りデータをメモリ内に保持することです  
cmd -> kafka(pyspark-topic1) <- spark streaming(df) -> メモリ

In [13]:
memory_sink = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("aggregates") \
  .option("checkpointLocation", "/tmp/kafka/memory/") \
  .start()

21/12/12 02:09:52 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [24]:
spark.sql("select * from aggregates").show()

# メモリシンクについてはまだ使い道があるので、のちのセクションで改めて紹介します。

+---+---------+
|key|    value|
+---+---------+
| 11|111111111|
|  2|        2|
|222|      111|
|333|     333,|
| aa|     1111|
+---+---------+



In [18]:
#　同時にサブスクライブもできる
memory_sink2 = df \
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)") \
  .writeStream \
  .format("memory") \
  .queryName("aggregates2") \
  .start()

21/12/12 02:11:48 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-72b67146-1e6d-4791-ac8e-3487aff5b6ea. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/12/12 02:11:48 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [21]:
spark.sql("select * from aggregates2").show()

+---+-----+
|key|value|
+---+-----+
| aa| 1111|
+---+-----+



In [25]:
memory_sink.stop()
memory_sink2.stop()

In [26]:
spark.stop()

# 各シンクの使い分け

1. コンソールシンク  
コンソールシンクは、デバッグ用として主に利用することが多いです
2. ファイルシンク  
一度ストレージ(S3やHDFS)に保存して後でバッチ処理を行う
3. kafkaシンク  
別のシステムとの連携や複雑な計算や変換を行った後の中間データとして配置する
4. メモリシンク  
リアルタイムでデータを集計したりするために使う

今回データはcmdから送信していましたが、次のセクションは一歩進んでPythonからデータを送信しつつAvro形式でのやり取りをみていこうと思います。